<div style="background-color: rgb(230, 216, 173); padding: 10px; height: 70px; border-radius: 15px;">
    <div style="font-family: 'Georgia', serif; font-size: 20px; padding: 10px; text-align: right; position: absolute; right: 20px; color: #0a0a0a;">
        <span style="color: Black;">Neel Adalja</span> <br>
        <span style="font-family: 'Arial', sans-serif; font-size: 12px; color: #0a0a0a;">Data Scientist | Data Analyst</span>
    </div>
</div>

<div style="background-color:hsl(184, 33.00%, 56.10%); padding: 10px; text-align: center; color: black; font-size: 32px; font-family: 'Arial', sans-serif;">
    Face Recognition System <br>
    <h3 style="text-align: center; color: black; font-size: 15px; font-family: 'Arial', sans-serif;">Python | OpenCV </h3>
</div>

<div style="background-color: lightgreen; color: black; padding: 4px;">
    <h3> PROJECT DESCRIPTION 
</h3> </div>

In [359]:
!pip install opencv-python
!pip install tensorflow







In [362]:
# Import essential libraries in the notebook

import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import MobileNetV2


In [364]:
# Initialize the webcam using OpenCV

cap = cv2.VideoCapture(0)

In [366]:
# Capture frames

ret, frame = cap.read()

Save the captured images for labeled classes:
- Display the live feed.
- Use key events (`q`,`s`) to quit or save the frame.

In [390]:
# code for capturing and saving images

import cv2
import os

# Create directories for each person (label)
label = "Binayak Lamichhanec"  # Change this for different classes
output_dir = f"./data/{label}"
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(0)
print("Press 's' to save the image, 'q' to quit.")

image_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow("Webcam", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('s'):  # Save the frame
        image_path = os.path.join(output_dir, f"{label}_{image_count}.jpg")
        cv2.imwrite(image_path, frame)
        image_count += 1
        print(f"Saved: {image_path}")
    
    elif key == ord('q'):  # Quit the capture
        break

cap.release()
cv2.destroyAllWindows()

Press 's' to save the image, 'q' to quit.


**2. Detect and Crop Faces**

In [370]:
# Path to the directory where images are saved
image_dir = "./data/Binayak Lamichhanec"

In [371]:
# Get the latest saved image from the directory
images = sorted(os.listdir(image_dir), key=lambda x: os.path.getctime(os.path.join(image_dir, x)))
if len(images) == 0:
    print("No images found in the directory.")
    exit()

latest_image_path = os.path.join(image_dir, images[-1])
print(f"Using image: {latest_image_path}")

Using image: ./data/Binayak Lamichhanec\Binayak Lamichhanec_3.jpg


In [375]:
# Read the latest image
image = cv2.imread(latest_image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [377]:
# Load the Haar Cascade face detection model using OpenCV.
# This model will be used to detect faces in the captured images.

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [379]:
# Perform face detection
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

In [381]:
# Perform face detection with adjusted parameters
faces = face_cascade.detectMultiScale(
    gray,
    scaleFactor=1.05,  # Reduce the scale step for finer detection
    minNeighbors=6,    # Increase for stricter face detection
    minSize=(50, 50)   # Minimum face size to detect
)

In [383]:
# Draw bounding boxes around detected faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)

`cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)` 
- faces is a list of tuples where each tuple represents a detected face.
- (x, y) - Coordinates of the top-left corner of the bounding box.
- w and h - Width and Height of the bounding box.
- (255, 0, 0),2: The color of the rectangle (in BGR format, this is blue) with 2 thickness

Example: `faces = [(50, 50, 100, 100), (200, 150, 80, 80)]`

For the first face:
- (x, y) = (50, 50)
- (x + w, y + h) = (150, 150)

For the second face:
- (x, y) = (200, 150)
- (x + w, y + h) = (280, 230)

In [385]:
# Display the image with bounding boxes
cv2.imshow("Detected Faces", image)
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()

In [387]:

# Draw bounding boxes and add labels
if len(faces) == 0:
    print("No faces detected.")
else:
    for (x, y, w, h) in faces:
        # Draw rectangle with custom color and thickness
        color = (0, 255, 0)  # Green color for the box
        thickness = 2
        cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness)
        
        # Add label above the rectangle
        label = "Face"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        font_thickness = 1
        label_size, _ = cv2.getTextSize(label, font, font_scale, font_thickness)
        
        label_x = x
        label_y = y - 10 if y - 10 > 10 else y + 10  # Adjust position if label goes out of bounds
        cv2.rectangle(image, (label_x, label_y - label_size[1] - 2), (label_x + label_size[0], label_y + 2), color, cv2.FILLED)
        cv2.putText(image, label, (label_x, label_y), font, font_scale, (0, 0, 0), font_thickness)

# Display the image with bounding boxes and labels
cv2.imshow("Detected Faces", image)
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()

**Preprocess and Crop Faces**

- Load saved images
- Detect faces in each image using Haar Cascade
- Crop the face regions for further processing

In [353]:
output_dir = './processed_faces/'  # Directory to save preprocessed faces
os.makedirs(output_dir, exist_ok=True)

if len(faces) == 0:
    print("No faces detected.")
else:
    # Loop through detected faces
    for idx, (x, y, w, h) in enumerate(faces):
        # Crop the face
        cropped_face = image[y:y+h, x:x+w]
        
        # Resize to a uniform size (e.g., 160x160 for models like FaceNet)
        resized_face = cv2.resize(cropped_face, (160, 160))
        
        # Save the preprocessed face
        face_path = os.path.join(output_dir, f"face_{idx}.jpg")
        cv2.imwrite(face_path, resized_face)
        print(f"Saved preprocessed face: {face_path}")
        
        # Display the preprocessed face (optional)
        cv2.imshow("Preprocessed Face", resized_face)
        cv2.waitKey(0) # The 0 argument tells OpenCV to wait indefinitely for the user to press any key

cv2.destroyAllWindows()

Saved preprocessed face: ./processed_faces/face_0.jpg


**3. Generate Face Embeddings**

Generating face embeddings is about converting a face image into a numerical representation (a vector of numbers) that *captures its unique features*.

- Input: You give the model a preprocessed face image (e.g., cropped and resized).

- Model Processing: The model analyzes the facial features, like the shape of the eyes, nose, and mouth.

- Output (Embedding): The model outputs a set of numbers (embedding) that uniquely represents the face, making it easy to compare with other faces for tasks like recognition or clustering.

Think of it as turning a face into a "digital fingerprint" for machines to understand.








In [360]:
# Load a pre-trained model

from tensorflow.keras.applications import MobileNetV2

model = MobileNetV2(weights='imagenet', 
                    include_top=False, 
                    input_shape=(160, 160, 3))

# If you have downloaded a model
model = load_model('model.h5')  

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# Initialize lists for embeddings and labels
embeddings = []
labels = []

In [ ]:
# define the folder for preprocessed faces
processed_faces_dir = './processed_faces'

# Generate embeddings for all preprocessed faces

# Iterate through all images in the directory
for image_name in os.listdir(processed_faces_dir):
    image_path = os.path.join(processed_faces_dir, image_name)

    # Check if the file is a valid image (optional: you can adjust based on file types)
    if image_name.endswith('.jpg') or image_name.endswith('.jpeg') or image_name.endswith('.png'):
        # Load and preprocess image
        face_image = cv2.imread(image_path)
        face_image = cv2.resize(face_image, (160, 160))  # Ensure correct input size
        face_preprocessed = face_image.astype('float32') / 255.0  # Normalize
        
        # Generate embedding
        embedding = model.predict(np.expand_dims(face_preprocessed, axis=0))[0]
        embeddings.append(embedding)
        
        # Use the image filename as the label (without file extension)
        label = os.path.splitext(image_name)[0]
        labels.append(label)

print(f"Generated embeddings for {len(embeddings)} faces.")

In [ ]:
labels

**4. Train a Classifier**

- Collect embeddings and corresponding labels.
- Train a KNN model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Train a KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(embeddings, labels)
print("Classifier trained.")


Test the system

In [309]:
# Test on a new face
new_face = cv2.imread('./processed_faces/test_face.jpg')
new_embedding = get_embedding(facenet_model, new_face)

# Predict the identity
predicted_label = knn.predict([new_embedding])
print("Predicted Identity:", predicted_label[0])


NameError: name 'get_embedding' is not defined

<div style="background-color: #002147; color: #fff; padding: 30px; text-align: center;">
    <h1>THANK YOU!
        <!--- Ram | Tech Skills Trainer | AI/ML Consultant --->
</h1> </div>

<h2 style="background-color: rgb(230, 216, 173); color: #2f4f4f; padding: 10px; border-radius: 10px; width: 350px; text-align: center; float: right; margin: 20px 0;">
    Neel Adalja<br>
    <span style="font-size: 12px; color:rgb(0, 0, 0);">
        Data Analyst | Data Scientist
    </span>
</h2>

<!--- Mohammad Idrees Bhat | Tech Skills Trainer | AI/ML Consultant --->